<a href="https://colab.research.google.com/github/AbrarHossainHimself/AbrarHossainHimself/blob/main/UCR_prog_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This tool captures all possible function call sequences for a given Python program. By analyzing the Abstract Syntax Tree (AST) of the program, it generates all potential call sequences that could occur during actual execution. The tool outputs a structured representation of these call paths, which can aid in understanding complex call flows, especially in recursive or interdependent function environments.

## Structure

The tool is organized into three main components:

1. **FunctionCallAnalyzer**: This class builds a call graph by visiting each function definition and call expression in the source code. Using `ast.NodeVisitor`, it identifies and stores the relationships between functions in the program.
  
2. **CallSequenceGenerator**: Based on the call graph, this class recursively generates all possible call sequences up to a specified depth (`max_depth`). It ensures each unique sequence is captured without redundancy, even in cases of recursion or mutual recursion.
  
3. **analyze_python_code**: This function orchestrates the process. It takes Python source code as input, parses it to create an AST, builds the call graph using `FunctionCallAnalyzer`, and generates all possible call sequences for each function using `CallSequenceGenerator`. The final output is a dictionary of unique sequences for each function, providing a detailed representation of all possible execution paths.

## Usage

To analyze a Python program's call sequences, use the `analyze_python_code` function. This function accepts a string containing the source code and returns a dictionary, where each key is a function name, and the value is a list of lists representing unique call sequences from that function.

### Example

```python
# Sample source code
source_code = """
def main():
    foo()
    bar()

def foo():
    baz()

def bar():
    baz()
    qux()

def baz():
    pass

def qux():
    baz()
"""

# Generate call sequences
sequences = analyze_python_code(source_code)

# Print the sequences
for function, call_sequences in sequences.items():
    print(f"\nPossible call sequences starting from '{function}':")
    for seq in call_sequences:
        print(" -> ".join(seq))
```

### Expected Output

The output will list all possible function call paths, starting from each function and showing the various sequences of calls that could occur. For example:

```
Possible call sequences starting from 'main':
main -> foo -> baz
main -> bar -> baz
main -> bar -> qux -> baz
```

## Implementation Details

- **AST Parsing**: The `ast.parse` function is used to convert source code into an AST, which allows structured traversal and inspection of function definitions and calls.
- **Recursive Call Generation**: The `CallSequenceGenerator` uses a recursive approach to explore all possible paths, limiting recursion depth to `max_depth` to prevent infinite loops in the case of deep or cyclic recursion.
- **Unique Sequence Filtering**: Duplicate call sequences are filtered out, ensuring that each call path is unique and free from redundancy.

## Constraints

- The tool currently handles functions referenced directly by name.
- Built-in and external library functions are included in sequences if directly called; however, they are not resolved to external definitions.
- Class methods and nested functions are not analyzed unless extended to do so.


In [ ]:
import ast
from typing import Set, List, Dict
from collections import defaultdict

class FunctionCallAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.call_graph = defaultdict(list)
        self.current_function = None
        self.all_functions = set()

    def visit_FunctionDef(self, node):
        self.current_function = node.name
        self.all_functions.add(node.name)
        # Reset calls for this function
        self.call_graph[node.name] = []
        # Visit all nodes in the function body
        for stmt in node.body:
            self.visit(stmt)
        # Reset current_function after processing
        self.current_function = None

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            if self.current_function:
                # Only add if it's not already in the list
                if node.func.id not in self.call_graph[self.current_function]:
                    self.call_graph[self.current_function].append(node.func.id)
        self.generic_visit(node)

class CallSequenceGenerator:
    def __init__(self, call_graph: Dict[str, List[str]], all_functions: Set[str]):
        self.call_graph = call_graph
        self.all_functions = all_functions
        self.max_depth = 10

    def generate_sequences(self, start_function: str) -> List[List[str]]:
        sequences = []
        visited = set()
        self._generate_sequences(start_function, [], sequences, visited, 0)
        return sequences

    def _generate_sequences(self, current_function: str, current_path: List[str],
                            sequences: List[List[str]], visited: Set[str], depth: int):
        """Generate sequences recursively."""
        if depth > self.max_depth or current_function not in self.all_functions:
            return

        # Add current function to path
        current_sequence = current_path + [current_function]
        sequences.append(current_sequence[:])

        # Get all direct function calls from current function
        if current_function in self.call_graph:
            for called_function in self.call_graph[current_function]:
                # Create unique call signature
                call_sig = f"{current_function}->{called_function}"
                if call_sig not in visited and depth < self.max_depth:
                    visited.add(call_sig)
                    self._generate_sequences(called_function, current_sequence,
                                             sequences, visited, depth + 1)
                    visited.remove(call_sig)

def analyze_python_code(source_code: str) -> Dict[str, List[List[str]]]:
    # Parse code
    tree = ast.parse(source_code)

    # Analyze AST
    analyzer = FunctionCallAnalyzer()
    analyzer.visit(tree)

    # Generate call sequences for each function
    generator = CallSequenceGenerator(analyzer.call_graph, analyzer.all_functions)

    # Generate and store sequences for each function
    all_sequences = {}
    for function in analyzer.all_functions:
        sequences = generator.generate_sequences(function)
        # Remove duplicates while preserving order
        unique_sequences = []
        seen = set()
        for seq in sequences:
            seq_tuple = tuple(seq)
            if seq_tuple not in seen:
                seen.add(seq_tuple)
                unique_sequences.append(seq)
        all_sequences[function] = unique_sequences

    return all_sequences


In [ ]:
# Test Cases

test_cases = {
    "Test Case 1": """
def start():
    a()
    b()

def a():
    c()

def b():
    c()

def c():
    pass
""",
    "Test Case 2": """
def factorial(n):
    if n > 1:
        return n * factorial(n - 1)
    else:
        return 1
""",
    "Test Case 3": """
def is_even(n):
    if n == 0:
        return True
    else:
        return is_odd(n - 1)

def is_odd(n):
    if n == 0:
        return False
    else:
        return is_even(n - 1)
""",
    "Test Case 4": """
def standalone():
    pass
""",
    "Test Case 5": """
def outer():
    def inner():
        pass
    inner()
""",
    "Test Case 6": """
def func_a():
    pass

def func_b():
    pass

def caller():
    func = func_a
    func()
""",
    "Test Case 7": """
def main():
    try:
        risky_function()
    except Exception as e:
        handle_error(e)

def risky_function():
    raise ValueError("An error occurred")

def handle_error(e):
    pass
""",
    "Test Case 8": """
class MyClass:
    def method_a(self):
        self.method_b()

    def method_b(self):
        pass

def start():
    obj = MyClass()
    obj.method_a()
""",
    "Test Case 9": """
def compute():
    step1()
    step2()
    step1()

def step1():
    pass

def step2():
    pass
"""
}


In [ ]:
selected_test_case = "Test Case 9"  # Change this to select a different test case

# Get source code
source_code = test_cases[selected_test_case]

# Analyze Python code
sequences = analyze_python_code(source_code)

# Print results
print(f"Analyzing {selected_test_case}:\n")
for function, call_sequences in sorted(sequences.items()):
    print(f"\nPossible call sequences starting from '{function}':")
    for seq in call_sequences:
        print(" -> ".join(seq))


Analyzing Test Case 9:


Possible call sequences starting from 'compute':
compute
compute -> step1
compute -> step2

Possible call sequences starting from 'step1':
step1

Possible call sequences starting from 'step2':
step2


 # Example: Call graph captures an actual call sequence

```python
def a():
    if condition():  # Runtime condition
        b()
    else:
        c()

def b():
    d()

def c():
    d()

def d():
    print("d")

def condition():
    return True  # Could be True or False
```

### Static Call Graph
The static call graph shows all possible relationships:
- `a -> b`
- `a -> c`
- `b -> d`
- `c -> d`

### Actual Execution Paths
The actual execution will only follow **one** path depending on the runtime condition in `condition()`:
- **If `condition()` is `True`:** `a -> b -> d`
- **If `condition()` is `False`:** `a -> c -> d`


The static call graph captures that `a` could call either `b` or `c`, but in any single execution, only one of these calls will actually happen.

# Implementation with specific function calls

In [2]:
import ast
from typing import Set, List, Dict, Optional, Any
from collections import defaultdict
from dataclasses import dataclass
from enum import Enum

class CallContext(Enum):
    SEQUENTIAL = "sequential"
    EXCLUSIVE = "exclusive"
    CONDITIONAL = "conditional"
    LOOP = "loop"

@dataclass
class CallInfo:
    caller: str
    callee: str
    context: CallContext
    line_number: int
    path_conditions: List[str]

class CallGraphNode:
    def __init__(self, name: str):
        self.name = name
        self.calls: Dict[CallContext, List[CallInfo]] = {
            context: [] for context in CallContext
        }

    def add_call(self, call_info: CallInfo):
        self.calls[call_info.context].append(call_info)

class FunctionCallAnalyzer(ast.NodeVisitor):
    def __init__(self):
        self.call_graph: Dict[str, CallGraphNode] = {}
        self.current_function: Optional[str] = None
        self.all_functions: Set[str] = set()
        self.context_stack: List[CallContext] = [CallContext.SEQUENTIAL]
        self.condition_stack: List[str] = []

    @property
    def current_context(self) -> CallContext:
        return self.context_stack[-1] if self.context_stack else CallContext.SEQUENTIAL

    def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
        self.all_functions.add(node.name)
        previous_function = self.current_function
        self.current_function = node.name

        if node.name not in self.call_graph:
            self.call_graph[node.name] = CallGraphNode(node.name)

        self.generic_visit(node)
        self.current_function = previous_function

    def visit_If(self, node: ast.If) -> None:
        condition_str = ast.unparse(node.test)

        # First visit the condition
        self.visit(node.test)

        # Handle if branch
        self.condition_stack.append(condition_str)
        self.context_stack.append(CallContext.EXCLUSIVE)
        for stmt in node.body:
            self.visit(stmt)
        self.context_stack.pop()
        self.condition_stack.pop()

        # Handle else branch
        if node.orelse:
            self.condition_stack.append(f"not ({condition_str})")
            self.context_stack.append(CallContext.EXCLUSIVE)
            for stmt in node.orelse:
                self.visit(stmt)
            self.context_stack.pop()
            self.condition_stack.pop()

    def visit_Call(self, node: ast.Call) -> None:
        if isinstance(node.func, ast.Name) and self.current_function:
            callee = node.func.id

            # Determine the context
            context = self.current_context
            if callee == "condition" and context == CallContext.EXCLUSIVE:
                context = CallContext.SEQUENTIAL

            call_info = CallInfo(
                caller=self.current_function,
                callee=callee,
                context=context,
                line_number=node.lineno,
                path_conditions=self.condition_stack.copy()
            )

            self.call_graph[self.current_function].add_call(call_info)

        self.generic_visit(node)

def generate_complete_sequences(start: str, call_graph: Dict[str, CallGraphNode],
                             all_functions: Set[str], max_depth: int = 10) -> Dict[str, List[Dict[str, Any]]]:
    sequences = defaultdict(list)
    visited = set()

    def _generate_sequences(current: str, path: List[str],
                          conditions: List[str], context: CallContext,
                          depth: int) -> None:
        if depth > max_depth or current not in call_graph:
            return

        current_path = path + [current]

        if current in call_graph:
            node = call_graph[current]

            # First, add the current sequence
            if len(current_path) > 1:  # Only add if it's not just the start node
                sequences[context.value].append({
                    "sequence": current_path,
                    "conditions": conditions
                })

            # Then continue with child calls
            for ctx, calls in node.calls.items():
                for call in calls:
                    call_sig = f"{current}->{call.callee}"
                    if call_sig not in visited and depth < max_depth:
                        visited.add(call_sig)

                        # Determine the context for the extended sequence
                        new_context = ctx
                        if context == CallContext.EXCLUSIVE:
                            new_context = CallContext.EXCLUSIVE

                        # Combine conditions
                        new_conditions = conditions + call.path_conditions

                        _generate_sequences(
                            call.callee,
                            current_path,
                            new_conditions,
                            new_context,
                            depth + 1
                        )
                        visited.remove(call_sig)

    _generate_sequences(start, [], [], CallContext.SEQUENTIAL, 0)
    return dict(sequences)

def analyze_python_code(source_code: str) -> Dict[str, Any]:
    # Parse code
    tree = ast.parse(source_code)

    # Analyze AST
    analyzer = FunctionCallAnalyzer()
    analyzer.visit(tree)

    # Process results
    all_sequences = {}
    call_relationships = {}

    # Process call relationships
    for function in analyzer.all_functions:
        node = analyzer.call_graph.get(function)
        if node:
            relationships = {}
            for context in CallContext:
                calls = [call.callee for call in node.calls[context]]
                if calls:
                    relationships[f"{context.value}_calls"] = calls
            if relationships:
                call_relationships[function] = relationships

    # Generate complete sequences for each function
    for function in analyzer.all_functions:
        sequences = generate_complete_sequences(
            function,
            analyzer.call_graph,
            analyzer.all_functions
        )
        if sequences:
            all_sequences[function] = sequences

    return {
        "sequences": all_sequences,
        "relationships": call_relationships
    }



In [3]:
# Test code
if __name__ == "__main__":
    test_code = """
def a():
    if condition():  # Runtime condition
        b()
    else:
        c()

def b():
    d()

def c():
    d()

def d():
    print("d")

def condition():
    return True  # Could be True or False
    """

    result = analyze_python_code(test_code)

    print("\n__________Call Relationships:__________")
    for func, rels in result["relationships"].items():
        print(f"\nFunction: {func}")
        for rel_type, calls in rels.items():
            if calls:
                print(f"  {rel_type}: {calls}")

    print("\n__________Detailed Sequences:_________")
    for func, sequences in result["sequences"].items():
        print(f"\nFunction: {func}")
        for context, seqs in sequences.items():
            if seqs:
                print(f"  {context}:")
                for seq in seqs:
                    print(f"    Sequence: {' -> '.join(seq['sequence'])}")
                    if seq['conditions']:
                        print(f"    Conditions: {seq['conditions']}")


__________Call Relationships:__________

Function: d
  sequential_calls: ['print']

Function: b
  sequential_calls: ['d']

Function: a
  sequential_calls: ['condition']
  exclusive_calls: ['b', 'c']

Function: c
  sequential_calls: ['d']

__________Detailed Sequences:_________

Function: b
  sequential:
    Sequence: b -> d

Function: a
  sequential:
    Sequence: a -> condition
  exclusive:
    Sequence: a -> b
    Conditions: ['condition()']
    Sequence: a -> b -> d
    Conditions: ['condition()']
    Sequence: a -> c
    Conditions: ['not (condition())']
    Sequence: a -> c -> d
    Conditions: ['not (condition())']

Function: c
  sequential:
    Sequence: c -> d
